In [2]:
#Completing imports:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def getcontours(img):
    # Finding all contours to store them in the contours variable:
    contours, hierarchy= cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    rectangles = []
    # Area Threshold:
    for contour in contours:
        if 35< cv2.contourArea(contour)<600:
            x, y, w, h = cv2.boundingRect(contour)
            rectangles.append((x, y, w, h))

    # Defining these functions to eliminate multiple bounding boxes appearing on one or more player(s) very close together.
    # Calculates the distance between both boxes' centers and eliminates them according to threshold value which can be fine-tuned.
    def get_center(rect):
        x, y, w, h = rect
        center_x = x + w / 2
        center_y = y + h / 2
        return (center_x, center_y)

    def distance(c1, c2):
        return np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2)

    threshold = 100

    
    # Finally, initialising a list to store rectangles that are not too close to others:
    filtered_rectangles = []
    to_remove = set()  # Set to track the rectangles to be removed.

    # Compare each rectangle to others
    for i, rect1 in enumerate(rectangles):
        rect1_center = get_center(rect1)
        for j, rect2 in enumerate(rectangles):
            if i != j:  # Do not compare the rectangle with itself.
                rect2_center = get_center(rect2)
                if distance(rect1_center, rect2_center) < threshold:
                    to_remove.add(i)
                    to_remove.add(j)

    # Keeping only the rectangles that are not in the "to_remove" set
    filtered_rectangles = [rect for i, rect in enumerate(rectangles) if i not in to_remove]

    # Creating bounding boxes over these filtered_rectangles:
    for rect in filtered_rectangles:
        x, y, w, h = rect
        cv2.rectangle(imgcontour, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [4]:
# Opening the video file:
video_path = 'volleyball_match.mp4'
cap = cv2.VideoCapture(video_path)

# Checking if the video was opened successfully or not:
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    while True:
        # Reading frames from the video
        ret, img = cap.read()

        #Creating a copy of the frame to create the final bounding boxes on. This image will eventually be displayed:
        imgcontour=img.copy()
        
        # Drawing a rectangle over the spare ball area to avoid false positives.
        # Identified the coordinates and optimised them after multiple rounds of trial and error:
        cv2.rectangle(img,(450,70) , (532,132),(0,0,0),-1)
        
        #Converting the frame to HSV for colour masking:
        hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        
        

        # Creating a colour mask for the volleyball:
        lower_yellow=np.array([10,90,90])
        upper_yellow=np.array([38,255,255])

        maskyellow=cv2.inRange(hsv,lower_yellow,upper_yellow)
        resyellow=cv2.bitwise_and(img,img,mask=maskyellow)

        # Eroding the image to eliminate tiny specks of noise like shoes:        
        kernel=np.ones((3,3),np.uint8)
        eroded=cv2.erode(maskyellow, kernel , iterations=2)

        # Dilation, connected different parts of the players' bodies so as to make the area filter more fruitful.
        # Used a different kernel size here for a more powerful dilation:
        kernel2=np.ones((5,5),np.uint8)
        dilated=cv2.dilate(eroded, kernel2 , iterations=2)
        
        
        
        #cv2.imshow("Dilated", dilated)
        #cv2.imshow("Res", resyellow)
        #cv2.imshow("Eroded", eroded)
        #cv2.imshow("Dilated", dilated)

        # Finally performing the contour operations on this dilated image:
        getcontours(dilated)
        cv2.imshow('Img Contour',imgcontour)
        
        #cv2.imshow("Image", img)
        

        # Wait for the appropriate time to respect the frame rate (video given is 45 fps, which makes it roughly 22 ms between frames) :
        key = cv2.waitKey(10) & 0xFF
        
        # Video Exit Mechanism:
        if key == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()